UJI COBA Data Independen 2025
PADA BAB 6

In [ ]:
# UJI DATA INDEPENDEN 2025 (JAN–NOV)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf
import pickle
import os
from sklearn.metrics import mean_absolute_percentage_error

NAMA_WILAYAH = "Surabaya" 
path_scaler = r"E:\SKRIPSI 2025\dataset\FINAL\Surabaya_w7_e114_b64\Surabaya_scaler_W7_E114_B64_DO0.02.pkl"
path_model = r"E:\SKRIPSI 2025\dataset\FINAL\Surabaya_w7_e114_b64\Surabaya_model_W7_E114_B64_DO0.02.h5"

path_data_2025 = rf"E:\Skripsi\Prediksi-bawang-merah\python\data\data clean\Data_Clean_Surabaya_2025.xlsx"

path_output_folder = r"E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025"
os.makedirs(path_output_folder, exist_ok=True)

WINDOW_SIZE = 7
def create_sliding_window(dataset, window_size):
    X, Y = [], []
    for i in range(len(dataset) - window_size):
        X.append(dataset[i:(i + window_size), 0])
        Y.append(dataset[i + window_size, 0])
    return np.array(X), np.array(Y)

print("="*60)
print(f"UJI DATA INDEPENDEN 2025 (JAN–NOV)")
print(f"WILAYAH : {NAMA_WILAYAH}")
print("="*60)

# 1. Load model & scaler
print("1. Memuat model dan scaler...")
model = tf.keras.models.load_model(path_model)
with open(path_scaler, 'rb') as f:
    scaler = pickle.load(f)

# 2. Load data 2025 (DATA ASLI)
print("2. Membaca data 2025...")
df_2025 = pd.read_excel(path_data_2025)

# Pastikan kolom tanggal
if 'Tanggal' in df_2025.columns:
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])
else:
    df_2025.rename(columns={df_2025.columns[0]: 'Tanggal'}, inplace=True)
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])

df_2025 = df_2025.sort_values('Tanggal')

# Kolom harga
col_harga = 'Harga (Rp)' if 'Harga (Rp)' in df_2025.columns else df_2025.columns[-1]

data_asli = df_2025[col_harga].values.reshape(-1, 1)
tanggal_full = df_2025['Tanggal'].values

print(f"   → Total data: {len(data_asli)} baris")

# 3. Normalisasi (HANYA untuk input model)
scaled_data = scaler.transform(data_asli)

# 4. Sliding window
X_test, _ = create_sliding_window(scaled_data, WINDOW_SIZE)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

_, Y_test_asli = create_sliding_window(data_asli, WINDOW_SIZE)
tanggal_uji = tanggal_full[WINDOW_SIZE:]

# 5. Prediksi
print("3. Melakukan prediksi...")
prediksi_scaled = model.predict(X_test, verbose=0)
prediksi_asli = scaler.inverse_transform(prediksi_scaled)

# 6. Hitung MAPE (DATA ASLI vs HASIL DENORMALISASI)
mape = mean_absolute_percentage_error(
    Y_test_asli.flatten(),
    prediksi_asli.flatten()
) * 100

print("\nHASIL PENGUJIAN")
print("-"*40)
print(f"Jumlah Data Uji : {len(Y_test_asli)} hari")
print(f"MAPE            : {mape:.4f}%")
print("-"*40)

plt.figure(figsize=(12, 6))

# Plot Data Aktual
plt.plot(
    tanggal_uji,
    Y_test_asli.flatten(),
    label='Harga Aktual (Data Asli)',
    color='#0077b6',
    linewidth=2
)


plt.plot(
    tanggal_uji,
    prediksi_asli.flatten(),
    label='Harga Prediksi (Model LSTM)',
    color='#ff6f00',
    linewidth=2,
    linestyle='-'
)

plt.title(
    f'Validasi Data Independen 2025\n'
    f'Wilayah {NAMA_WILAYAH} (MAPE: {mape:.2f}%)',
    fontsize=14,
    fontweight='bold'
)

plt.xlabel('Bulan (Tahun 2025)')
plt.ylabel('Harga Bawang Merah (Rp)')

ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()

plot_path = os.path.join(
    path_output_folder,
    f"Grafik_Prediksi_{NAMA_WILAYAH}_2025.png"
)
plt.savefig(plot_path, dpi=300)
plt.close()


df_hasil = pd.DataFrame({
    'Tanggal': tanggal_uji,
    'Harga Aktual (Rp)': Y_test_asli.flatten(),
    'Harga Prediksi (Rp)': prediksi_asli.flatten(),
    'Error Absolut': abs(Y_test_asli.flatten() - prediksi_asli.flatten())
})

excel_path = os.path.join(
    path_output_folder,
    f"Hasil_Prediksi_{NAMA_WILAYAH}_2025.xlsx"
)
df_hasil.to_excel(excel_path, index=False)

print("\nOUTPUT TERSIMPAN:")
print(f"✓ Grafik : {plot_path}")
print(f"✓ Excel  : {excel_path}")
print("✓ Proses uji data independen selesai.")

UJI DATA INDEPENDEN 2025 (JAN–NOV)
WILAYAH : Surabaya
1. Memuat model dan scaler...
2. Membaca data 2025...
   → Total data: 238 baris
3. Melakukan prediksi...

HASIL PENGUJIAN
----------------------------------------
Jumlah Data Uji : 231 hari
MAPE            : 2.3243%
----------------------------------------

OUTPUT TERSIMPAN:
✓ Grafik : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Grafik_Prediksi_Surabaya_2025.png
✓ Excel  : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Hasil_Prediksi_Surabaya_2025.xlsx
✓ Proses uji data independen selesai.


In [6]:
# UJI DATA INDEPENDEN 2025 (JAN–NOV)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf
import pickle
import os
from sklearn.metrics import mean_absolute_percentage_error

NAMA_WILAYAH = "Banyuwangi" 
path_scaler = r"E:\SKRIPSI 2025\dataset\FINAL\Banyuwangi_w7_e114_b64\Banyuwangi_scaler_W7_E114_B64_DO0.02.pkl"
path_model = r"E:\SKRIPSI 2025\dataset\FINAL\Banyuwangi_w7_e114_b64\Banyuwangi_model_W7_E114_B64_DO0.02.h5"

path_data_2025 = r"E:\Skripsi\Prediksi-bawang-merah\python\data\data clean\Data_Clean_Banyuwangi_2025.xlsx"

path_output_folder = r"E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025"
os.makedirs(path_output_folder, exist_ok=True)

WINDOW_SIZE = 7
def create_sliding_window(dataset, window_size):
    X, Y = [], []
    for i in range(len(dataset) - window_size):
        X.append(dataset[i:(i + window_size), 0])
        Y.append(dataset[i + window_size, 0])
    return np.array(X), np.array(Y)

print("="*60)
print(f"UJI DATA INDEPENDEN 2025 (JAN–NOV)")
print(f"WILAYAH : {NAMA_WILAYAH}")
print("="*60)

# 1. Load model & scaler
print("1. Memuat model dan scaler...")
model = tf.keras.models.load_model(path_model)
with open(path_scaler, 'rb') as f:
    scaler = pickle.load(f)

# 2. Load data 2025 (DATA ASLI)
print("2. Membaca data 2025...")
df_2025 = pd.read_excel(path_data_2025)

# Pastikan kolom tanggal
if 'Tanggal' in df_2025.columns:
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])
else:
    df_2025.rename(columns={df_2025.columns[0]: 'Tanggal'}, inplace=True)
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])

df_2025 = df_2025.sort_values('Tanggal')

# Kolom harga
col_harga = 'Harga (Rp)' if 'Harga (Rp)' in df_2025.columns else df_2025.columns[-1]

data_asli = df_2025[col_harga].values.reshape(-1, 1)
tanggal_full = df_2025['Tanggal'].values

print(f"   → Total data: {len(data_asli)} baris")

# 3. Normalisasi (HANYA untuk input model)
scaled_data = scaler.transform(data_asli)

# 4. Sliding window
X_test, _ = create_sliding_window(scaled_data, WINDOW_SIZE)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

_, Y_test_asli = create_sliding_window(data_asli, WINDOW_SIZE)
tanggal_uji = tanggal_full[WINDOW_SIZE:]

# 5. Prediksi
print("3. Melakukan prediksi...")
prediksi_scaled = model.predict(X_test, verbose=0)
prediksi_asli = scaler.inverse_transform(prediksi_scaled)

# 6. Hitung MAPE (DATA ASLI vs HASIL DENORMALISASI)
mape = mean_absolute_percentage_error(
    Y_test_asli.flatten(),
    prediksi_asli.flatten()
) * 100

print("\nHASIL PENGUJIAN")
print("-"*40)
print(f"Jumlah Data Uji : {len(Y_test_asli)} hari")
print(f"MAPE            : {mape:.4f}%")
print("-"*40)

plt.figure(figsize=(12, 6))

# Plot Data Aktual
plt.plot(
    tanggal_uji,
    Y_test_asli.flatten(),
    label='Harga Aktual (Data Asli)',
    color='#0077b6',
    linewidth=2
)


plt.plot(
    tanggal_uji,
    prediksi_asli.flatten(),
    label='Harga Prediksi (Model LSTM)',
    color='#ff6f00',
    linewidth=2,
    linestyle='-'
)

plt.title(
    f'Validasi Data Independen 2025\n'
    f'Wilayah {NAMA_WILAYAH} (MAPE: {mape:.2f}%)',
    fontsize=14,
    fontweight='bold'
)

plt.xlabel('Bulan (Tahun 2025)')
plt.ylabel('Harga Bawang Merah (Rp)')

ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()

plot_path = os.path.join(
    path_output_folder,
    f"Grafik_Prediksi_{NAMA_WILAYAH}_2025.png"
)
plt.savefig(plot_path, dpi=300)
plt.close()


df_hasil = pd.DataFrame({
    'Tanggal': tanggal_uji,
    'Harga Aktual (Rp)': Y_test_asli.flatten(),
    'Harga Prediksi (Rp)': prediksi_asli.flatten(),
    'Error Absolut': abs(Y_test_asli.flatten() - prediksi_asli.flatten())
})

excel_path = os.path.join(
    path_output_folder,
    f"Hasil_Prediksi_{NAMA_WILAYAH}_2025.xlsx"
)
df_hasil.to_excel(excel_path, index=False)

print("\nOUTPUT TERSIMPAN:")
print(f"✓ Grafik : {plot_path}")
print(f"✓ Excel  : {excel_path}")
print("✓ Proses uji data independen selesai.")

UJI DATA INDEPENDEN 2025 (JAN–NOV)
WILAYAH : Banyuwangi
1. Memuat model dan scaler...
2. Membaca data 2025...
   → Total data: 238 baris
3. Melakukan prediksi...

HASIL PENGUJIAN
----------------------------------------
Jumlah Data Uji : 231 hari
MAPE            : 3.7341%
----------------------------------------

OUTPUT TERSIMPAN:
✓ Grafik : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Grafik_Prediksi_Banyuwangi_2025.png
✓ Excel  : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Hasil_Prediksi_Banyuwangi_2025.xlsx
✓ Proses uji data independen selesai.


In [7]:
# UJI DATA INDEPENDEN 2025 (JAN–NOV)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf
import pickle
import os
from sklearn.metrics import mean_absolute_percentage_error

NAMA_WILAYAH = "Jember" 
path_scaler = r"E:\SKRIPSI 2025\dataset\FINAL\Jember_w7_e114_b64\Jember_scaler_W7_E114_B64_DO0.02.pkl"
path_model = r"E:\SKRIPSI 2025\dataset\FINAL\Jember_w7_e114_b64\Jember_model_W7_E114_B64_DO0.02.h5"

path_data_2025 = r"E:\Skripsi\Prediksi-bawang-merah\python\data\data clean\Data_Clean_Jember_2025.xlsx"

path_output_folder = r"E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025"
os.makedirs(path_output_folder, exist_ok=True)

WINDOW_SIZE = 7
def create_sliding_window(dataset, window_size):
    X, Y = [], []
    for i in range(len(dataset) - window_size):
        X.append(dataset[i:(i + window_size), 0])
        Y.append(dataset[i + window_size, 0])
    return np.array(X), np.array(Y)

print("="*60)
print(f"UJI DATA INDEPENDEN 2025 (JAN–NOV)")
print(f"WILAYAH : {NAMA_WILAYAH}")
print("="*60)

# 1. Load model & scaler
print("1. Memuat model dan scaler...")
model = tf.keras.models.load_model(path_model)
with open(path_scaler, 'rb') as f:
    scaler = pickle.load(f)

# 2. Load data 2025 (DATA ASLI)
print("2. Membaca data 2025...")
df_2025 = pd.read_excel(path_data_2025)

# Pastikan kolom tanggal
if 'Tanggal' in df_2025.columns:
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])
else:
    df_2025.rename(columns={df_2025.columns[0]: 'Tanggal'}, inplace=True)
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])

df_2025 = df_2025.sort_values('Tanggal')

# Kolom harga
col_harga = 'Harga (Rp)' if 'Harga (Rp)' in df_2025.columns else df_2025.columns[-1]

data_asli = df_2025[col_harga].values.reshape(-1, 1)
tanggal_full = df_2025['Tanggal'].values

print(f"   → Total data: {len(data_asli)} baris")

# 3. Normalisasi (HANYA untuk input model)
scaled_data = scaler.transform(data_asli)

# 4. Sliding window
X_test, _ = create_sliding_window(scaled_data, WINDOW_SIZE)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

_, Y_test_asli = create_sliding_window(data_asli, WINDOW_SIZE)
tanggal_uji = tanggal_full[WINDOW_SIZE:]

# 5. Prediksi
print("3. Melakukan prediksi...")
prediksi_scaled = model.predict(X_test, verbose=0)
prediksi_asli = scaler.inverse_transform(prediksi_scaled)

# 6. Hitung MAPE (DATA ASLI vs HASIL DENORMALISASI)
mape = mean_absolute_percentage_error(
    Y_test_asli.flatten(),
    prediksi_asli.flatten()
) * 100

print("\nHASIL PENGUJIAN")
print("-"*40)
print(f"Jumlah Data Uji : {len(Y_test_asli)} hari")
print(f"MAPE            : {mape:.4f}%")
print("-"*40)

plt.figure(figsize=(12, 6))

# Plot Data Aktual
plt.plot(
    tanggal_uji,
    Y_test_asli.flatten(),
    label='Harga Aktual (Data Asli)',
    color='#0077b6',
    linewidth=2
)


plt.plot(
    tanggal_uji,
    prediksi_asli.flatten(),
    label='Harga Prediksi (Model LSTM)',
    color='#ff6f00',
    linewidth=2,
    linestyle='-'
)

plt.title(
    f'Validasi Data Independen 2025\n'
    f'Wilayah {NAMA_WILAYAH} (MAPE: {mape:.2f}%)',
    fontsize=14,
    fontweight='bold'
)

plt.xlabel('Bulan (Tahun 2025)')
plt.ylabel('Harga Bawang Merah (Rp)')

ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()

plot_path = os.path.join(
    path_output_folder,
    f"Grafik_Prediksi_{NAMA_WILAYAH}_2025.png"
)
plt.savefig(plot_path, dpi=300)
plt.close()


df_hasil = pd.DataFrame({
    'Tanggal': tanggal_uji,
    'Harga Aktual (Rp)': Y_test_asli.flatten(),
    'Harga Prediksi (Rp)': prediksi_asli.flatten(),
    'Error Absolut': abs(Y_test_asli.flatten() - prediksi_asli.flatten())
})

excel_path = os.path.join(
    path_output_folder,
    f"Hasil_Prediksi_{NAMA_WILAYAH}_2025.xlsx"
)
df_hasil.to_excel(excel_path, index=False)

print("\nOUTPUT TERSIMPAN:")
print(f"✓ Grafik : {plot_path}")
print(f"✓ Excel  : {excel_path}")
print("✓ Proses uji data independen selesai.")

UJI DATA INDEPENDEN 2025 (JAN–NOV)
WILAYAH : Jember
1. Memuat model dan scaler...
2. Membaca data 2025...
   → Total data: 238 baris
3. Melakukan prediksi...

HASIL PENGUJIAN
----------------------------------------
Jumlah Data Uji : 231 hari
MAPE            : 2.2940%
----------------------------------------

OUTPUT TERSIMPAN:
✓ Grafik : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Grafik_Prediksi_Jember_2025.png
✓ Excel  : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Hasil_Prediksi_Jember_2025.xlsx
✓ Proses uji data independen selesai.


In [12]:
# UJI DATA INDEPENDEN 2025 (JAN–NOV)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf
import pickle
import os
from sklearn.metrics import mean_absolute_percentage_error

NAMA_WILAYAH = "Blitar" 
path_scaler = r"E:\SKRIPSI 2025\dataset\FINAL\Blitar_w7_e114_b64\Blitar_scaler_W7_E114_B64_DO0.02.pkl"
path_model = r"E:\SKRIPSI 2025\dataset\FINAL\Blitar_w7_e114_b64\Blitar_model_W7_E114_B64_DO0.02.h5"

path_data_2025 = r"E:\Skripsi\Prediksi-bawang-merah\python\data\data clean\Data_Clean_Kota_Blitar_2025.xlsx"

path_output_folder = r"E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025"
os.makedirs(path_output_folder, exist_ok=True)

WINDOW_SIZE = 7
def create_sliding_window(dataset, window_size):
    X, Y = [], []
    for i in range(len(dataset) - window_size):
        X.append(dataset[i:(i + window_size), 0])
        Y.append(dataset[i + window_size, 0])
    return np.array(X), np.array(Y)

print("="*60)
print(f"UJI DATA INDEPENDEN 2025 (JAN–NOV)")
print(f"WILAYAH : {NAMA_WILAYAH}")
print("="*60)

# 1. Load model & scaler
print("1. Memuat model dan scaler...")
model = tf.keras.models.load_model(path_model)
with open(path_scaler, 'rb') as f:
    scaler = pickle.load(f)

# 2. Load data 2025 (DATA ASLI)
print("2. Membaca data 2025...")
df_2025 = pd.read_excel(path_data_2025)

# Pastikan kolom tanggal
if 'Tanggal' in df_2025.columns:
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])
else:
    df_2025.rename(columns={df_2025.columns[0]: 'Tanggal'}, inplace=True)
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])

df_2025 = df_2025.sort_values('Tanggal')

# Kolom harga
col_harga = 'Harga (Rp)' if 'Harga (Rp)' in df_2025.columns else df_2025.columns[-1]

data_asli = df_2025[col_harga].values.reshape(-1, 1)
tanggal_full = df_2025['Tanggal'].values

print(f"   → Total data: {len(data_asli)} baris")

# 3. Normalisasi (HANYA untuk input model)
scaled_data = scaler.transform(data_asli)

# 4. Sliding window
X_test, _ = create_sliding_window(scaled_data, WINDOW_SIZE)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

_, Y_test_asli = create_sliding_window(data_asli, WINDOW_SIZE)
tanggal_uji = tanggal_full[WINDOW_SIZE:]

# 5. Prediksi
print("3. Melakukan prediksi...")
prediksi_scaled = model.predict(X_test, verbose=0)
prediksi_asli = scaler.inverse_transform(prediksi_scaled)

# 6. Hitung MAPE (DATA ASLI vs HASIL DENORMALISASI)
mape = mean_absolute_percentage_error(
    Y_test_asli.flatten(),
    prediksi_asli.flatten()
) * 100

print("\nHASIL PENGUJIAN")
print("-"*40)
print(f"Jumlah Data Uji : {len(Y_test_asli)} hari")
print(f"MAPE            : {mape:.4f}%")
print("-"*40)

plt.figure(figsize=(12, 6))

# Plot Data Aktual
plt.plot(
    tanggal_uji,
    Y_test_asli.flatten(),
    label='Harga Aktual (Data Asli)',
    color='#0077b6',
    linewidth=2
)


plt.plot(
    tanggal_uji,
    prediksi_asli.flatten(),
    label='Harga Prediksi (Model LSTM)',
    color='#ff6f00',
    linewidth=2,
    linestyle='-'
)

plt.title(
    f'Validasi Data Independen 2025\n'
    f'Wilayah {NAMA_WILAYAH} (MAPE: {mape:.2f}%)',
    fontsize=14,
    fontweight='bold'
)

plt.xlabel('Bulan (Tahun 2025)')
plt.ylabel('Harga Bawang Merah (Rp)')

ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()

plot_path = os.path.join(
    path_output_folder,
    f"Grafik_Prediksi_{NAMA_WILAYAH}_2025.png"
)
plt.savefig(plot_path, dpi=300)
plt.close()


df_hasil = pd.DataFrame({
    'Tanggal': tanggal_uji,
    'Harga Aktual (Rp)': Y_test_asli.flatten(),
    'Harga Prediksi (Rp)': prediksi_asli.flatten(),
    'Error Absolut': abs(Y_test_asli.flatten() - prediksi_asli.flatten())
})

excel_path = os.path.join(
    path_output_folder,
    f"Hasil_Prediksi_{NAMA_WILAYAH}_2025.xlsx"
)
df_hasil.to_excel(excel_path, index=False)

print("\nOUTPUT TERSIMPAN:")
print(f"✓ Grafik : {plot_path}")
print(f"✓ Excel  : {excel_path}")
print("✓ Proses uji data independen selesai.")

UJI DATA INDEPENDEN 2025 (JAN–NOV)
WILAYAH : Blitar
1. Memuat model dan scaler...
2. Membaca data 2025...
   → Total data: 238 baris
3. Melakukan prediksi...

HASIL PENGUJIAN
----------------------------------------
Jumlah Data Uji : 231 hari
MAPE            : 3.6886%
----------------------------------------

OUTPUT TERSIMPAN:
✓ Grafik : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Grafik_Prediksi_Blitar_2025.png
✓ Excel  : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Hasil_Prediksi_Blitar_2025.xlsx
✓ Proses uji data independen selesai.


In [13]:
# UJI DATA INDEPENDEN 2025 (JAN–NOV)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf
import pickle
import os
from sklearn.metrics import mean_absolute_percentage_error

NAMA_WILAYAH = "Kediri" 
path_scaler = r"E:\SKRIPSI 2025\dataset\FINAL\Kediri_w7_e114_b64\Kediri_scaler_W7_E114_B64_DO0.02.pkl"
path_model = r"E:\SKRIPSI 2025\dataset\FINAL\Kediri_w7_e114_b64\Kediri_model_W7_E114_B64_DO0.02.h5"

path_data_2025 = r"E:\Skripsi\Prediksi-bawang-merah\python\data\data clean\Data_Clean_Kota_Kediri_2025.xlsx"

path_output_folder = r"E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025"
os.makedirs(path_output_folder, exist_ok=True)

WINDOW_SIZE = 7
def create_sliding_window(dataset, window_size):
    X, Y = [], []
    for i in range(len(dataset) - window_size):
        X.append(dataset[i:(i + window_size), 0])
        Y.append(dataset[i + window_size, 0])
    return np.array(X), np.array(Y)

print("="*60)
print(f"UJI DATA INDEPENDEN 2025 (JAN–NOV)")
print(f"WILAYAH : {NAMA_WILAYAH}")
print("="*60)

# 1. Load model & scaler
print("1. Memuat model dan scaler...")
model = tf.keras.models.load_model(path_model)
with open(path_scaler, 'rb') as f:
    scaler = pickle.load(f)

# 2. Load data 2025 (DATA ASLI)
print("2. Membaca data 2025...")
df_2025 = pd.read_excel(path_data_2025)

# Pastikan kolom tanggal
if 'Tanggal' in df_2025.columns:
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])
else:
    df_2025.rename(columns={df_2025.columns[0]: 'Tanggal'}, inplace=True)
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])

df_2025 = df_2025.sort_values('Tanggal')

# Kolom harga
col_harga = 'Harga (Rp)' if 'Harga (Rp)' in df_2025.columns else df_2025.columns[-1]

data_asli = df_2025[col_harga].values.reshape(-1, 1)
tanggal_full = df_2025['Tanggal'].values

print(f"   → Total data: {len(data_asli)} baris")

# 3. Normalisasi (HANYA untuk input model)
scaled_data = scaler.transform(data_asli)

# 4. Sliding window
X_test, _ = create_sliding_window(scaled_data, WINDOW_SIZE)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

_, Y_test_asli = create_sliding_window(data_asli, WINDOW_SIZE)
tanggal_uji = tanggal_full[WINDOW_SIZE:]

# 5. Prediksi
print("3. Melakukan prediksi...")
prediksi_scaled = model.predict(X_test, verbose=0)
prediksi_asli = scaler.inverse_transform(prediksi_scaled)

# 6. Hitung MAPE (DATA ASLI vs HASIL DENORMALISASI)
mape = mean_absolute_percentage_error(
    Y_test_asli.flatten(),
    prediksi_asli.flatten()
) * 100

print("\nHASIL PENGUJIAN")
print("-"*40)
print(f"Jumlah Data Uji : {len(Y_test_asli)} hari")
print(f"MAPE            : {mape:.4f}%")
print("-"*40)

plt.figure(figsize=(12, 6))

# Plot Data Aktual
plt.plot(
    tanggal_uji,
    Y_test_asli.flatten(),
    label='Harga Aktual (Data Asli)',
    color='#0077b6',
    linewidth=2
)


plt.plot(
    tanggal_uji,
    prediksi_asli.flatten(),
    label='Harga Prediksi (Model LSTM)',
    color='#ff6f00',
    linewidth=2,
    linestyle='-'
)

plt.title(
    f'Validasi Data Independen 2025\n'
    f'Wilayah {NAMA_WILAYAH} (MAPE: {mape:.2f}%)',
    fontsize=14,
    fontweight='bold'
)

plt.xlabel('Bulan (Tahun 2025)')
plt.ylabel('Harga Bawang Merah (Rp)')

ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()

plot_path = os.path.join(
    path_output_folder,
    f"Grafik_Prediksi_{NAMA_WILAYAH}_2025.png"
)
plt.savefig(plot_path, dpi=300)
plt.close()


df_hasil = pd.DataFrame({
    'Tanggal': tanggal_uji,
    'Harga Aktual (Rp)': Y_test_asli.flatten(),
    'Harga Prediksi (Rp)': prediksi_asli.flatten(),
    'Error Absolut': abs(Y_test_asli.flatten() - prediksi_asli.flatten())
})

excel_path = os.path.join(
    path_output_folder,
    f"Hasil_Prediksi_{NAMA_WILAYAH}_2025.xlsx"
)
df_hasil.to_excel(excel_path, index=False)

print("\nOUTPUT TERSIMPAN:")
print(f"✓ Grafik : {plot_path}")
print(f"✓ Excel  : {excel_path}")
print("✓ Proses uji data independen selesai.")

UJI DATA INDEPENDEN 2025 (JAN–NOV)
WILAYAH : Kediri
1. Memuat model dan scaler...
2. Membaca data 2025...
   → Total data: 238 baris
3. Melakukan prediksi...

HASIL PENGUJIAN
----------------------------------------
Jumlah Data Uji : 231 hari
MAPE            : 2.1739%
----------------------------------------

OUTPUT TERSIMPAN:
✓ Grafik : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Grafik_Prediksi_Kediri_2025.png
✓ Excel  : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Hasil_Prediksi_Kediri_2025.xlsx
✓ Proses uji data independen selesai.


In [14]:
# UJI DATA INDEPENDEN 2025 (JAN–NOV)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf
import pickle
import os
from sklearn.metrics import mean_absolute_percentage_error

NAMA_WILAYAH = "Madiun" 
path_scaler = r"E:\SKRIPSI 2025\dataset\FINAL\Madiun_w7_e114_b64\Madiun_scaler_W7_E114_B64_DO0.02.pkl"
path_model = r"E:\SKRIPSI 2025\dataset\FINAL\Madiun_w7_e114_b64\Madiun_model_W7_E114_B64_DO0.02.h5"

path_data_2025 = r"E:\Skripsi\Prediksi-bawang-merah\python\data\data clean\Data_Clean_Kota_Madiun_2025.xlsx"

path_output_folder = r"E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025"
os.makedirs(path_output_folder, exist_ok=True)

WINDOW_SIZE = 7
def create_sliding_window(dataset, window_size):
    X, Y = [], []
    for i in range(len(dataset) - window_size):
        X.append(dataset[i:(i + window_size), 0])
        Y.append(dataset[i + window_size, 0])
    return np.array(X), np.array(Y)

print("="*60)
print(f"UJI DATA INDEPENDEN 2025 (JAN–NOV)")
print(f"WILAYAH : {NAMA_WILAYAH}")
print("="*60)

# 1. Load model & scaler
print("1. Memuat model dan scaler...")
model = tf.keras.models.load_model(path_model)
with open(path_scaler, 'rb') as f:
    scaler = pickle.load(f)

# 2. Load data 2025 (DATA ASLI)
print("2. Membaca data 2025...")
df_2025 = pd.read_excel(path_data_2025)

# Pastikan kolom tanggal
if 'Tanggal' in df_2025.columns:
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])
else:
    df_2025.rename(columns={df_2025.columns[0]: 'Tanggal'}, inplace=True)
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])

df_2025 = df_2025.sort_values('Tanggal')

# Kolom harga
col_harga = 'Harga (Rp)' if 'Harga (Rp)' in df_2025.columns else df_2025.columns[-1]

data_asli = df_2025[col_harga].values.reshape(-1, 1)
tanggal_full = df_2025['Tanggal'].values

print(f"   → Total data: {len(data_asli)} baris")

# 3. Normalisasi (HANYA untuk input model)
scaled_data = scaler.transform(data_asli)

# 4. Sliding window
X_test, _ = create_sliding_window(scaled_data, WINDOW_SIZE)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

_, Y_test_asli = create_sliding_window(data_asli, WINDOW_SIZE)
tanggal_uji = tanggal_full[WINDOW_SIZE:]

# 5. Prediksi
print("3. Melakukan prediksi...")
prediksi_scaled = model.predict(X_test, verbose=0)
prediksi_asli = scaler.inverse_transform(prediksi_scaled)

# 6. Hitung MAPE (DATA ASLI vs HASIL DENORMALISASI)
mape = mean_absolute_percentage_error(
    Y_test_asli.flatten(),
    prediksi_asli.flatten()
) * 100

print("\nHASIL PENGUJIAN")
print("-"*40)
print(f"Jumlah Data Uji : {len(Y_test_asli)} hari")
print(f"MAPE            : {mape:.4f}%")
print("-"*40)

plt.figure(figsize=(12, 6))

# Plot Data Aktual
plt.plot(
    tanggal_uji,
    Y_test_asli.flatten(),
    label='Harga Aktual (Data Asli)',
    color='#0077b6',
    linewidth=2
)


plt.plot(
    tanggal_uji,
    prediksi_asli.flatten(),
    label='Harga Prediksi (Model LSTM)',
    color='#ff6f00',
    linewidth=2,
    linestyle='-'
)

plt.title(
    f'Validasi Data Independen 2025\n'
    f'Wilayah {NAMA_WILAYAH} (MAPE: {mape:.2f}%)',
    fontsize=14,
    fontweight='bold'
)

plt.xlabel('Bulan (Tahun 2025)')
plt.ylabel('Harga Bawang Merah (Rp)')

ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()

plot_path = os.path.join(
    path_output_folder,
    f"Grafik_Prediksi_{NAMA_WILAYAH}_2025.png"
)
plt.savefig(plot_path, dpi=300)
plt.close()


df_hasil = pd.DataFrame({
    'Tanggal': tanggal_uji,
    'Harga Aktual (Rp)': Y_test_asli.flatten(),
    'Harga Prediksi (Rp)': prediksi_asli.flatten(),
    'Error Absolut': abs(Y_test_asli.flatten() - prediksi_asli.flatten())
})

excel_path = os.path.join(
    path_output_folder,
    f"Hasil_Prediksi_{NAMA_WILAYAH}_2025.xlsx"
)
df_hasil.to_excel(excel_path, index=False)

print("\nOUTPUT TERSIMPAN:")
print(f"✓ Grafik : {plot_path}")
print(f"✓ Excel  : {excel_path}")
print("✓ Proses uji data independen selesai.")

UJI DATA INDEPENDEN 2025 (JAN–NOV)
WILAYAH : Madiun
1. Memuat model dan scaler...
2. Membaca data 2025...
   → Total data: 238 baris
3. Melakukan prediksi...

HASIL PENGUJIAN
----------------------------------------
Jumlah Data Uji : 231 hari
MAPE            : 5.2563%
----------------------------------------

OUTPUT TERSIMPAN:
✓ Grafik : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Grafik_Prediksi_Madiun_2025.png
✓ Excel  : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Hasil_Prediksi_Madiun_2025.xlsx
✓ Proses uji data independen selesai.


In [15]:
# UJI DATA INDEPENDEN 2025 (JAN–NOV)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf
import pickle
import os
from sklearn.metrics import mean_absolute_percentage_error

NAMA_WILAYAH = "Malang" 
path_scaler = r"E:\SKRIPSI 2025\dataset\FINAL\Malang_w7_e114_b64\Malang_scaler_W7_E114_B64_DO0.02.pkl"
path_model = r"E:\SKRIPSI 2025\dataset\FINAL\Malang_w7_e114_b64\Malang_model_W7_E114_B64_DO0.02.h5"

path_data_2025 = r"E:\Skripsi\Prediksi-bawang-merah\python\data\data clean\Data_Clean_Kota_Malang_2025.xlsx"

path_output_folder = r"E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025"
os.makedirs(path_output_folder, exist_ok=True)

WINDOW_SIZE = 7
def create_sliding_window(dataset, window_size):
    X, Y = [], []
    for i in range(len(dataset) - window_size):
        X.append(dataset[i:(i + window_size), 0])
        Y.append(dataset[i + window_size, 0])
    return np.array(X), np.array(Y)

print("="*60)
print(f"UJI DATA INDEPENDEN 2025 (JAN–NOV)")
print(f"WILAYAH : {NAMA_WILAYAH}")
print("="*60)

# 1. Load model & scaler
print("1. Memuat model dan scaler...")
model = tf.keras.models.load_model(path_model)
with open(path_scaler, 'rb') as f:
    scaler = pickle.load(f)

# 2. Load data 2025 (DATA ASLI)
print("2. Membaca data 2025...")
df_2025 = pd.read_excel(path_data_2025)

# Pastikan kolom tanggal
if 'Tanggal' in df_2025.columns:
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])
else:
    df_2025.rename(columns={df_2025.columns[0]: 'Tanggal'}, inplace=True)
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])

df_2025 = df_2025.sort_values('Tanggal')

# Kolom harga
col_harga = 'Harga (Rp)' if 'Harga (Rp)' in df_2025.columns else df_2025.columns[-1]

data_asli = df_2025[col_harga].values.reshape(-1, 1)
tanggal_full = df_2025['Tanggal'].values

print(f"   → Total data: {len(data_asli)} baris")

# 3. Normalisasi (HANYA untuk input model)
scaled_data = scaler.transform(data_asli)

# 4. Sliding window
X_test, _ = create_sliding_window(scaled_data, WINDOW_SIZE)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

_, Y_test_asli = create_sliding_window(data_asli, WINDOW_SIZE)
tanggal_uji = tanggal_full[WINDOW_SIZE:]

# 5. Prediksi
print("3. Melakukan prediksi...")
prediksi_scaled = model.predict(X_test, verbose=0)
prediksi_asli = scaler.inverse_transform(prediksi_scaled)

# 6. Hitung MAPE (DATA ASLI vs HASIL DENORMALISASI)
mape = mean_absolute_percentage_error(
    Y_test_asli.flatten(),
    prediksi_asli.flatten()
) * 100

print("\nHASIL PENGUJIAN")
print("-"*40)
print(f"Jumlah Data Uji : {len(Y_test_asli)} hari")
print(f"MAPE            : {mape:.4f}%")
print("-"*40)

plt.figure(figsize=(12, 6))

# Plot Data Aktual
plt.plot(
    tanggal_uji,
    Y_test_asli.flatten(),
    label='Harga Aktual (Data Asli)',
    color='#0077b6',
    linewidth=2
)


plt.plot(
    tanggal_uji,
    prediksi_asli.flatten(),
    label='Harga Prediksi (Model LSTM)',
    color='#ff6f00',
    linewidth=2,
    linestyle='-'
)

plt.title(
    f'Validasi Data Independen 2025\n'
    f'Wilayah {NAMA_WILAYAH} (MAPE: {mape:.2f}%)',
    fontsize=14,
    fontweight='bold'
)

plt.xlabel('Bulan (Tahun 2025)')
plt.ylabel('Harga Bawang Merah (Rp)')

ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()

plot_path = os.path.join(
    path_output_folder,
    f"Grafik_Prediksi_{NAMA_WILAYAH}_2025.png"
)
plt.savefig(plot_path, dpi=300)
plt.close()


df_hasil = pd.DataFrame({
    'Tanggal': tanggal_uji,
    'Harga Aktual (Rp)': Y_test_asli.flatten(),
    'Harga Prediksi (Rp)': prediksi_asli.flatten(),
    'Error Absolut': abs(Y_test_asli.flatten() - prediksi_asli.flatten())
})

excel_path = os.path.join(
    path_output_folder,
    f"Hasil_Prediksi_{NAMA_WILAYAH}_2025.xlsx"
)
df_hasil.to_excel(excel_path, index=False)

print("\nOUTPUT TERSIMPAN:")
print(f"✓ Grafik : {plot_path}")
print(f"✓ Excel  : {excel_path}")
print("✓ Proses uji data independen selesai.")

UJI DATA INDEPENDEN 2025 (JAN–NOV)
WILAYAH : Malang
1. Memuat model dan scaler...
2. Membaca data 2025...
   → Total data: 238 baris
3. Melakukan prediksi...

HASIL PENGUJIAN
----------------------------------------
Jumlah Data Uji : 231 hari
MAPE            : 4.0936%
----------------------------------------

OUTPUT TERSIMPAN:
✓ Grafik : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Grafik_Prediksi_Malang_2025.png
✓ Excel  : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Hasil_Prediksi_Malang_2025.xlsx
✓ Proses uji data independen selesai.


In [16]:
# UJI DATA INDEPENDEN 2025 (JAN–NOV)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf
import pickle
import os
from sklearn.metrics import mean_absolute_percentage_error

NAMA_WILAYAH = "Probolinggo" 
path_scaler = r"E:\SKRIPSI 2025\dataset\FINAL\Probolinggo_w7_e114_b64\Probolinggo_scaler_W7_E114_B64_DO0.02.pkl"
path_model = r"E:\SKRIPSI 2025\dataset\FINAL\Probolinggo_w7_e114_b64\Probolinggo_model_W7_E114_B64_DO0.02.h5"

path_data_2025 = r"E:\Skripsi\Prediksi-bawang-merah\python\data\data clean\Data_Clean_Kota_Probolinggo_2025.xlsx"

path_output_folder = r"E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025"
os.makedirs(path_output_folder, exist_ok=True)

WINDOW_SIZE = 7
def create_sliding_window(dataset, window_size):
    X, Y = [], []
    for i in range(len(dataset) - window_size):
        X.append(dataset[i:(i + window_size), 0])
        Y.append(dataset[i + window_size, 0])
    return np.array(X), np.array(Y)

print("="*60)
print(f"UJI DATA INDEPENDEN 2025 (JAN–NOV)")
print(f"WILAYAH : {NAMA_WILAYAH}")
print("="*60)

# 1. Load model & scaler
print("1. Memuat model dan scaler...")
model = tf.keras.models.load_model(path_model)
with open(path_scaler, 'rb') as f:
    scaler = pickle.load(f)

# 2. Load data 2025 (DATA ASLI)
print("2. Membaca data 2025...")
df_2025 = pd.read_excel(path_data_2025)

# Pastikan kolom tanggal
if 'Tanggal' in df_2025.columns:
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])
else:
    df_2025.rename(columns={df_2025.columns[0]: 'Tanggal'}, inplace=True)
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])

df_2025 = df_2025.sort_values('Tanggal')

# Kolom harga
col_harga = 'Harga (Rp)' if 'Harga (Rp)' in df_2025.columns else df_2025.columns[-1]

data_asli = df_2025[col_harga].values.reshape(-1, 1)
tanggal_full = df_2025['Tanggal'].values

print(f"   → Total data: {len(data_asli)} baris")

# 3. Normalisasi (HANYA untuk input model)
scaled_data = scaler.transform(data_asli)

# 4. Sliding window
X_test, _ = create_sliding_window(scaled_data, WINDOW_SIZE)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

_, Y_test_asli = create_sliding_window(data_asli, WINDOW_SIZE)
tanggal_uji = tanggal_full[WINDOW_SIZE:]

# 5. Prediksi
print("3. Melakukan prediksi...")
prediksi_scaled = model.predict(X_test, verbose=0)
prediksi_asli = scaler.inverse_transform(prediksi_scaled)

# 6. Hitung MAPE (DATA ASLI vs HASIL DENORMALISASI)
mape = mean_absolute_percentage_error(
    Y_test_asli.flatten(),
    prediksi_asli.flatten()
) * 100

print("\nHASIL PENGUJIAN")
print("-"*40)
print(f"Jumlah Data Uji : {len(Y_test_asli)} hari")
print(f"MAPE            : {mape:.4f}%")
print("-"*40)

plt.figure(figsize=(12, 6))

# Plot Data Aktual
plt.plot(
    tanggal_uji,
    Y_test_asli.flatten(),
    label='Harga Aktual (Data Asli)',
    color='#0077b6',
    linewidth=2
)


plt.plot(
    tanggal_uji,
    prediksi_asli.flatten(),
    label='Harga Prediksi (Model LSTM)',
    color='#ff6f00',
    linewidth=2,
    linestyle='-'
)

plt.title(
    f'Validasi Data Independen 2025\n'
    f'Wilayah {NAMA_WILAYAH} (MAPE: {mape:.2f}%)',
    fontsize=14,
    fontweight='bold'
)

plt.xlabel('Bulan (Tahun 2025)')
plt.ylabel('Harga Bawang Merah (Rp)')

ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()

plot_path = os.path.join(
    path_output_folder,
    f"Grafik_Prediksi_{NAMA_WILAYAH}_2025.png"
)
plt.savefig(plot_path, dpi=300)
plt.close()


df_hasil = pd.DataFrame({
    'Tanggal': tanggal_uji,
    'Harga Aktual (Rp)': Y_test_asli.flatten(),
    'Harga Prediksi (Rp)': prediksi_asli.flatten(),
    'Error Absolut': abs(Y_test_asli.flatten() - prediksi_asli.flatten())
})

excel_path = os.path.join(
    path_output_folder,
    f"Hasil_Prediksi_{NAMA_WILAYAH}_2025.xlsx"
)
df_hasil.to_excel(excel_path, index=False)

print("\nOUTPUT TERSIMPAN:")
print(f"✓ Grafik : {plot_path}")
print(f"✓ Excel  : {excel_path}")
print("✓ Proses uji data independen selesai.")

UJI DATA INDEPENDEN 2025 (JAN–NOV)
WILAYAH : Probolinggo
1. Memuat model dan scaler...
2. Membaca data 2025...
   → Total data: 238 baris
3. Melakukan prediksi...

HASIL PENGUJIAN
----------------------------------------
Jumlah Data Uji : 231 hari
MAPE            : 1.9398%
----------------------------------------

OUTPUT TERSIMPAN:
✓ Grafik : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Grafik_Prediksi_Probolinggo_2025.png
✓ Excel  : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Hasil_Prediksi_Probolinggo_2025.xlsx
✓ Proses uji data independen selesai.


In [17]:
# UJI DATA INDEPENDEN 2025 (JAN–NOV)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf
import pickle
import os
from sklearn.metrics import mean_absolute_percentage_error

NAMA_WILAYAH = "Sumenep" 
path_scaler = r"E:\SKRIPSI 2025\dataset\FINAL\Sumenep_w7_e114_b64\Sumenep_scaler_W7_E114_B64_DO0.02.pkl"
path_model = r"E:\SKRIPSI 2025\dataset\FINAL\Sumenep_w7_e114_b64\Sumenep_model_W7_E114_B64_DO0.02.h5"

path_data_2025 = r"E:\Skripsi\Prediksi-bawang-merah\python\data\data clean\Data_Clean_Sumenep_2025.xlsx"

path_output_folder = r"E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025"
os.makedirs(path_output_folder, exist_ok=True)

WINDOW_SIZE = 7
def create_sliding_window(dataset, window_size):
    X, Y = [], []
    for i in range(len(dataset) - window_size):
        X.append(dataset[i:(i + window_size), 0])
        Y.append(dataset[i + window_size, 0])
    return np.array(X), np.array(Y)

print("="*60)
print(f"UJI DATA INDEPENDEN 2025 (JAN–NOV)")
print(f"WILAYAH : {NAMA_WILAYAH}")
print("="*60)

# 1. Load model & scaler
print("1. Memuat model dan scaler...")
model = tf.keras.models.load_model(path_model)
with open(path_scaler, 'rb') as f:
    scaler = pickle.load(f)

# 2. Load data 2025 (DATA ASLI)
print("2. Membaca data 2025...")
df_2025 = pd.read_excel(path_data_2025)

# Pastikan kolom tanggal
if 'Tanggal' in df_2025.columns:
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])
else:
    df_2025.rename(columns={df_2025.columns[0]: 'Tanggal'}, inplace=True)
    df_2025['Tanggal'] = pd.to_datetime(df_2025['Tanggal'])

df_2025 = df_2025.sort_values('Tanggal')

# Kolom harga
col_harga = 'Harga (Rp)' if 'Harga (Rp)' in df_2025.columns else df_2025.columns[-1]

data_asli = df_2025[col_harga].values.reshape(-1, 1)
tanggal_full = df_2025['Tanggal'].values

print(f"   → Total data: {len(data_asli)} baris")

# 3. Normalisasi (HANYA untuk input model)
scaled_data = scaler.transform(data_asli)

# 4. Sliding window
X_test, _ = create_sliding_window(scaled_data, WINDOW_SIZE)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

_, Y_test_asli = create_sliding_window(data_asli, WINDOW_SIZE)
tanggal_uji = tanggal_full[WINDOW_SIZE:]

# 5. Prediksi
print("3. Melakukan prediksi...")
prediksi_scaled = model.predict(X_test, verbose=0)
prediksi_asli = scaler.inverse_transform(prediksi_scaled)

# 6. Hitung MAPE (DATA ASLI vs HASIL DENORMALISASI)
mape = mean_absolute_percentage_error(
    Y_test_asli.flatten(),
    prediksi_asli.flatten()
) * 100

print("\nHASIL PENGUJIAN")
print("-"*40)
print(f"Jumlah Data Uji : {len(Y_test_asli)} hari")
print(f"MAPE            : {mape:.4f}%")
print("-"*40)

plt.figure(figsize=(12, 6))

# Plot Data Aktual
plt.plot(
    tanggal_uji,
    Y_test_asli.flatten(),
    label='Harga Aktual (Data Asli)',
    color='#0077b6',
    linewidth=2
)


plt.plot(
    tanggal_uji,
    prediksi_asli.flatten(),
    label='Harga Prediksi (Model LSTM)',
    color='#ff6f00',
    linewidth=2,
    linestyle='-'
)

plt.title(
    f'Validasi Data Independen 2025\n'
    f'Wilayah {NAMA_WILAYAH} (MAPE: {mape:.2f}%)',
    fontsize=14,
    fontweight='bold'
)

plt.xlabel('Bulan (Tahun 2025)')
plt.ylabel('Harga Bawang Merah (Rp)')

ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()

plot_path = os.path.join(
    path_output_folder,
    f"Grafik_Prediksi_{NAMA_WILAYAH}_2025.png"
)
plt.savefig(plot_path, dpi=300)
plt.close()


df_hasil = pd.DataFrame({
    'Tanggal': tanggal_uji,
    'Harga Aktual (Rp)': Y_test_asli.flatten(),
    'Harga Prediksi (Rp)': prediksi_asli.flatten(),
    'Error Absolut': abs(Y_test_asli.flatten() - prediksi_asli.flatten())
})

excel_path = os.path.join(
    path_output_folder,
    f"Hasil_Prediksi_{NAMA_WILAYAH}_2025.xlsx"
)
df_hasil.to_excel(excel_path, index=False)

print("\nOUTPUT TERSIMPAN:")
print(f"✓ Grafik : {plot_path}")
print(f"✓ Excel  : {excel_path}")
print("✓ Proses uji data independen selesai.")

UJI DATA INDEPENDEN 2025 (JAN–NOV)
WILAYAH : Sumenep
1. Memuat model dan scaler...
2. Membaca data 2025...
   → Total data: 238 baris
3. Melakukan prediksi...

HASIL PENGUJIAN
----------------------------------------
Jumlah Data Uji : 231 hari
MAPE            : 2.3640%
----------------------------------------

OUTPUT TERSIMPAN:
✓ Grafik : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Grafik_Prediksi_Sumenep_2025.png
✓ Excel  : E:\SKRIPSI 2025\dataset\INDEPENDENT DATASET 2025\Hasil_Prediksi_Sumenep_2025.xlsx
✓ Proses uji data independen selesai.
